In [296]:
import pandas as pd

import numpy as np
from scipy.stats import norm
import queue
from ctypes import string_at
from sys import getsizeof
import ctypes
from binascii import hexlify
%matplotlib inline
#以上操作导入包

In [297]:
df_train = pd.read_csv(r"C:\Users\fulian\Desktop\tarinset.csv",encoding='utf-8')
df_test = pd.read_csv(r"C:\Users\fulian\Desktop\test.csv",encoding='utf-8')
df_train = df_train.drop(df_train.columns[[0]], axis=1)
df_test = df_test.drop(df_test.columns[[0]], axis=1)

In [298]:
def featrue_split_list(df):
    c = []
    for i in range(0,df.shape[1] -1):
         featrues = set()
         for j in range(0,df.shape[0]):
            featrues.add(df.iat[j,i])
         a = list(featrues)
         
         c.append(a)
    
    return c
featrues = featrue_split_list(df_train)
l = (len(featrues))
print(featrues)

[[' 乌黑', ' 青绿', ' 浅白'], [' 蜷缩', ' 稍蜷', ' 硬挺'], [' 沉闷', ' 清脆', ' 浊响'], [' 稍糊', ' 清晰', ' 模糊'], [' 凹陷', ' 稍凹', ' 平坦'], [' 硬滑', ' 软粘']]


In [299]:
def B(x):
    return -x*np.log2(x)-(1-x)*np.log2(1-x)

In [300]:
class tree:
    def __init__(self,val):
        self.key = val
        self.left = None
        self.right = None
        self.middle = None
        self.root = val
    def insert(self,val):
      if isinstance(val,tree):
          if self.key != None:
             if self.left is None:
               self.left = val
            
             elif self.middle is None:
               self.middle = val
            
             elif self.right is None:
                self.right = val
          else:
            self.key = val
      else:
       if self.key != None:
         if self.left is None:
            self.left = tree(val)
            
         elif self.middle is None:
            self.middle = tree(val)
            
         elif self.right is None:
            self.right = tree(val)
              
         
       else:
         self.key = val
    



    def printValues(self):
       if self.left:
         self.left.printValues()
       print(self.key)
       if self.middle:
          self.middle.printValues()
       if self.right:
          self.right.printValues()
    def printlayer(self):
       q=queue.Queue(100)
       q.put(self)
       while q.empty() ==False:
          p = q.get()
          print(p.key)
          if p.left != None:
             q.put(p.left)
          if p.middle != None:
             q.put(p.middle)
          if p.right != None:
             q.put(p.right)
          
          

    


    



In [301]:
def all_split(df,a):
    i = 0
    for x in range(df.shape[0]):
        for j in range(df.shape[1]):
            if df.iat[x,j] == a:
                i = j
    data_split =  pd.DataFrame()
    for u in range(0,df.shape[0]):
         if df.iat[u,i] == a:
            data_split =data_split.append(df.iloc[u,:])
    return data_split   


     


In [302]:

listexp = [' 是',' 否']
def Ginichoose(parent,a):
    min = 1000
    best = 0
    for i in range(len(a)):
        sum2 = 0
        Gini = 0
        split = pd.DataFrame()
        for j in range(len(a[i])):
            sumsub = 0
            split = all_split(parent,a[i][j])
            for u in range(split.shape[0]):
                if split.iat[u,split.shape[1] - 1] == listexp[0]:
                    sumsub += 1
            if split.shape[0] != 0:
                sum2 = 1- (sumsub/split.shape[0])*(sumsub/split.shape[0])-((split.shape[0]-sumsub)/split.shape[0])*((split.shape[0]-sumsub)/split.shape[0])
            Gini = Gini + sum2*(split.shape[0]/parent.shape[0])
        
        if Gini < min :
            best = i
            min = Gini
    
    return a[best]


In [303]:
def delet(featrues_pre,featrues_deleted):
    f = -1
    for i in range(len(featrues_pre)):
        for j in range(len(featrues_pre[i])):
            if featrues_pre[i][j] == featrues_deleted[0]:
                f =i
                break
    if f>=0:
        del featrues_pre[f]
      
    return featrues_pre
 


In [304]:
def same_class(df):
    for i in range(df.shape[0]):
        if df.iat[i,df.shape[1] - 1] != df.iat[0,df.shape[1] - 1]:
            return False
    return True

    

In [305]:
def mode(df):
    listresult =[' 是',' 否']
    sum1 = 0
    sum2 = 0
    list_mode = []
    for i in range(df.shape[0]):
        if df.iat[i,df.shape[1] - 1] == listresult[0]:
            
            sum1 += 1
        else:
            sum2 += 1
    if sum1 > sum2:
        list_mode.append(listresult[0])
    else:
        list_mode.append(listresult[1])  
    list_mode_result = pd.DataFrame(list_mode)
    return list_mode




In [306]:
def DTL(examples,featrues,parent_examples):
    if examples.empty == True:
        return parent_examples
    elif same_class(examples) == True:
        return mode(examples)
    elif len(featrues) == 0:
        return mode(examples)
    else:
        
        best = Ginichoose(examples,featrues)
        Tree = tree(best)
        root = Tree
        for q in range(len(best)):
            examplesub = all_split(examples,best[q])
            temp = []
            temp.append(best[q])
            subtree = DTL(examplesub,delet(featrues,best),mode(examples))
            
            if Tree.left == None:
                Tree.insert(temp)
                Tree = Tree.left

            elif Tree.middle == None:
                Tree.insert(temp)
                Tree = Tree.middle

            elif Tree.right == None:
                Tree.insert(temp)
                Tree = Tree.right
            Tree.insert(subtree)
            Tree = root  
            
        return Tree



In [307]:

print("未剪枝决策树")
DTL(df_train,featrue_split_list(df_train),None).printlayer()
# Full_tree = DTL(df_train,featrue_split_list(df_train),None)
# Full_tree.printlayer()


未剪枝决策树
[' 乌黑', ' 青绿', ' 浅白']
[' 乌黑']
[' 青绿']
[' 浅白']
[' 蜷缩', ' 稍蜷', ' 硬挺']
[' 沉闷', ' 清脆', ' 浊响']
[' 否']
[' 蜷缩']
[' 稍蜷']
[' 硬挺']
[' 沉闷']
[' 清脆']
[' 浊响']
[' 是']
[' 稍糊', ' 清晰', ' 模糊']
[' 是']
[' 否']
[' 否']
[' 是']
[' 稍糊']
[' 清晰']
[' 模糊']
[' 是']
[' 否']
[' 否']


In [308]:
global p 
p = []

In [309]:
def all_split_index(df,a):
    index = []
    for x in range(df.shape[0]):
        for j in range(df.shape[1]):
            if df.iat[x,j] == a:
                index.append(x)
    
    return index 


In [310]:
def if_pruning(df):
   
    if len(p) == 1:
        s = 0
        for i in range(len(p[0])):
            t1 = all_split(df,p[0][i])
            for j in range(t1.shape[0]):
                if t1.iat[j,t1.shape[1] - 1] == mode(t1)[0]:
                    s += 1
        ratio1 = s/df_train.shape[0]
    else:
        a = p[1]
        b = p[0]
        parent_split = all_split_index(df,a)
        s = 0
        for j in range(len(b)):
            child_split = all_split_index(df,b[j])
            t1 = all_split(df,a)
            t = all_split(t1,b[j])
            
            for i in range(df.shape[0]):
                if i in parent_split and i in child_split:
                    if df.iat[i,df.shape[1] - 1] == mode(t)[0]:
                        s += 1
            for i in range(df.shape[0]):
                if i in parent_split and i not in child_split:
                    if df.iat[i,df.shape[1] - 1] == mode(t)[0]:
                        s += 1
        ratio1 = s/df_train.shape[0]
    
    return ratio1

    

In [311]:

def DTLpre(examples,featrues,parent_examples,ratio0):
    if examples.empty == True:
        return parent_examples
    elif same_class(examples) == True:
        return mode(examples)
    elif len(featrues) == 0:
        return mode(examples)
    else:
        best = Ginichoose(examples,featrues)
        p.append(best)
        
        ratio = if_pruning(df_train)
        if ratio < ratio0:
            return mode(examples)
        p.clear()
        Tree = tree(best)
        root = Tree
        for q in range(len(best)):
            examplesub = all_split(examples,best[q])
            p.append(best[q])
            subtree = DTLpre(examplesub,delet(featrues,best),mode(examples),ratio)
            
            if Tree.left == None:
                Tree.insert(best[q])
                Tree = Tree.left

            elif Tree.middle == None:
                Tree.insert(best[q])
                Tree = Tree.middle

            elif Tree.right == None:
                Tree.insert(best[q])
                Tree = Tree.right
            
            Tree.insert(subtree)
            #p.clear()
            Tree = root  
        return Tree



In [312]:
print("预剪枝决策树")
DTLpre(df_train,featrue_split_list(df_train),None,3/7).printlayer()


预剪枝决策树
[' 乌黑', ' 青绿', ' 浅白']
 乌黑
 青绿
 浅白
[' 是']
[' 否']
[' 否']


In [313]:
p.clear()
global ltree 
ltree = []

In [314]:
def binaryTreePaths(root):
       
        def construct_paths(root, path):
            if root:
                path += str(root.key)
                if not root.left and not root.right:  # 当前节点是叶子节点
                    paths.append(path)  # 把路径加入到答案中
                else:
                    path += '->'  # 当前节点不是叶子节点，继续递归遍历
                    construct_paths(root.left, path)
                    construct_paths(root.middle, path)
                    construct_paths(root.right, path)

        paths = []
        construct_paths(root, '')
        return paths


In [315]:
def if_pruning_last(df):
    
    return

In [316]:
Full_tree = DTL(df_train,featrue_split_list(df_train),None)
a = binaryTreePaths(Full_tree)
print(a[1])
a[1].split('->')
s = a[1].split('->')
print(s[2])

[' 乌黑', ' 青绿', ' 浅白']->[' 乌黑']->[' 蜷缩', ' 稍蜷', ' 硬挺']->[' 稍蜷']->[' 稍糊', ' 清晰', ' 模糊']->[' 稍糊']->[' 是']
[' 蜷缩', ' 稍蜷', ' 硬挺']
